In [101]:
import requests
import time
import json
import pandas as pd
from bs4 import BeautifulSoup

Scrape the previous podcasts archive

In [2]:
url_past_podcasts = "https://podcast.ucsd.edu/CourseList.aspx?show=past"

In [3]:
req_obj = requests.get(url_past_podcasts)

In [4]:
text_podcast_archive = BeautifulSoup(req_obj.text)

In [21]:
a_eles = text_podcast_archive.find_all("a", class_="PodcastLink")

In [22]:
len(a_eles)

2149

In [55]:
podcast_archive = []

In [58]:
for quarter in text_podcast_archive.find_all("div", class_="quarter"):
  curr_quarter = quarter.find("h2").text.strip()
  for course in quarter.find_all("a", class_="PodcastLink"):
    course_code = course.text.split("-")[0].rstrip()
    if len(course_code.split()) > 2:
      split_codes = course_code.split()
      [podcast_archive.append([(test_str[i] + " " + test_str[i-1]), curr_quarter]) for i in range(0, len(test_str), 2)]
    else:
      podcast_archive.append([course_code, curr_quarter])

In [59]:
len(podcast_archive)

2269

In [ ]:
podcast_archive

In [76]:
podcast_archive[0][1][-2:] + "-" + str(int(podcast_archive[0][1][-2:])+1)

'19-20'

In [77]:
# if winter or spring strip last two chars and conver 0X-(last two chars) string
# else fall so convert to (last two chars)-0X
for podcast in podcast_archive:
  if "Fall" in podcast[1]:
    academic_year = podcast[1][-2:] + "-" + str(int(podcast_archive[0][1][-2:])+1)
  else:
    academic_year = str(int(podcast_archive[0][1][-2:])+1) + "-" + podcast[1][-2:]
  podcast.append(academic_year)

In [78]:
podcast_archive[0]

['BGGN 202', 'Fall 2019', '19-20']

In [79]:
#with open('podcast_archive.json', 'w', encoding='utf-8') as f:
# json.dump(podcast_archive, f, ensure_ascii=False, indent=4)

## Scrape the course catalog

In [41]:
catalog_front_url = "http://registrar.ucsd.edu/catalog/{}-{}/front/courses.html"
catalog_base_url  = "http://registrar.ucsd.edu/catalog/{}-{}/"

In [42]:
print(catalog_front_url.format(18, 19))

http://registrar.ucsd.edu/catalog/18-19/front/courses.html


In [97]:
catalog_records = []

In [98]:
for year in range(10,21):
  academic_year = "{}-{}".format(f"{year-1:02d}", year)
  front_req_obj    = requests.get("http://registrar.ucsd.edu/catalog/{}-{}/front/courses.html".format(f"{year-1:02d}", year))
  front_page       = BeautifulSoup(req_obj.text)
  for course_page in cat_page.find_all("a", text="courses"):
    href           = course_page["href"]
    code = href.split("/")[-1].split(".")[0]
    dept_page_url = "http://registrar.ucsd.edu/catalog/{}-{}/".format(f"{year-1:02d}", year) + href.replace("../","")
    course_req_obj = requests.get(dept_page_url)
    course_page    = BeautifulSoup(course_req_obj.text)
    department_code = course_page.find_all("h1")[-1].text
    for course in course_page.find_all("p", class_="course-name"):
      course_code    = course.text.split(".")[0]
      course         = [course_code, code, department_code, academic_year, dept_page_url]
      catalog_records.append(course)
    time.sleep(0.5)

In [102]:
len(catalog_records)

58328

In [104]:
#with open('catalog_records.json', 'w', encoding='utf-8') as f:
#  json.dump(catalog_records, f, ensure_ascii=False, indent=4)

In [25]:
front_req_obj = requests.get("http://regyearstrar.ucsd.edu/catalog/{}-{}/front/courses.html".format(18, 19))

In [27]:
cat_page = BeautifulSoup(req_obj.text)

In [35]:
href = cat_page.find("a", text="courses")["href"]

In [59]:
course_req_obj = requests.get("http://registrar.ucsd.edu/catalog/{}-{}/".format(18, 19) + href.replace("../",""))

In [60]:
course_page = BeautifulSoup(course_req_obj.text)

In [63]:
course_page.find_all("p", class_="course-name")[0].text.split(".")[0]

'AIP 97'

In [96]:
catalog_records[0]

['197',
 'AIP',
 'Academic Internship Program ',
 '09-10',
 'http://registrar.ucsd.edu/catalog/09-10/courses/AIP.html']

In [108]:
with open('catalog_records.json') as f:
  catalog_records = json.load(f)

In [109]:
for course in catalog_records:
  if course[0][:2].isnumeric():
    course[0] = course[1] + " " + course[0]

## Merge Datasets

In [120]:
catalog_df=pd.DataFrame(catalog_records, columns=["course_code", "code_prefix", "department_desc", "academic_year", "link"])

In [121]:
podcast_archive_df=pd.DataFrame(podcast_archive, columns=["course_code", "term", "academic_year"])

In [122]:
podcast_archive_df["podcast"]=True

In [126]:
merged_df = catalog_df.merge(podcast_archive_df, on=["course_code", "academic_year"], how="left")

In [128]:
merged_df["podcast"] = merged_df["podcast"].fillna(False)

In [129]:
merged_df

,course_code,code_prefix,department_desc,academic_year,link,term,podcast
0,AIP 197,AIP,Academic Internship Program,09-10,http://registrar.ucsd.edu/catalog/09-10/course...,NaN,False
1,AIP 197PR,AIP,Academic Internship Program,09-10,http://registrar.ucsd.edu/catalog/09-10/course...,NaN,False
2,ANTH 1,ANTH,Anthropology,09-10,http://registrar.ucsd.edu/catalog/09-10/course...,NaN,False
3,ANTH 2,ANTH,Anthropology,09-10,http://registrar.ucsd.edu/catalog/09-10/course...,NaN,False
4,ANTH 3,ANTH,Anthropology,09-10,http://registrar.ucsd.edu/catalog/09-10/course...,NaN,False
...,...,...,...,...,...,...,...
58353,WARR 11A,WARR,Warren College,19-20,http://registrar.ucsd.edu/catalog/19-20/course...,NaN,False
58354,WARR 11B,WARR,Warren College,19-20,http://registrar.ucsd.edu/catalog/19-20/course...,NaN,False
58355,WCWP 100,WARR,Warren College,19-20,http://registrar.ucsd.edu/catalog/19-20/course...,NaN,False
58356,WARR 189,WARR,Warren College,19-20,http://registrar.ucsd.edu/catalog/19-20/course...,NaN,False


In [132]:
merged_df.to_json("initial_results.json")

In [133]:
merged_df.to_csv("initial_results.csv", index=False)